In [3]:
# Jane Street Market Prediction - Model Development (Optimized Version)
import sys
sys.path.append('..')

import polars as pl
import numpy as np
import pandas as pd
from sklearn.model_selection import TimeSeriesSplit
from sklearn.metrics import r2_score
from sklearn.ensemble import HistGradientBoostingRegressor
from typing import Dict, List, Tuple
import matplotlib.pyplot as plt
import seaborn as sns

class ModelDevelopment:
    """
    Model development pipeline for market prediction using optimized Histogram-based Gradient Boosting
    """
    def __init__(self, df: pl.DataFrame):
        self.df = df
        self.feature_cols = [col for col in df.columns 
                           if col.startswith(('feature_', 'target_lag_', 
                                            'return_', 'symbol_', 'vol_'))]
        self.target_col = 'responder_6'
        print(f"Identified {len(self.feature_cols)} features")
        
    def prepare_validation_splits(self, n_splits: int = 5) -> List[Tuple]:
        """
        Create time-based validation splits
        """
        # Sort by time
        df_sorted = self.df.sort(['date_id', 'time_id'])
        
        # Create splits
        tscv = TimeSeriesSplit(n_splits=n_splits)
        splits = []
        
        for train_idx, val_idx in tscv.split(df_sorted):
            train_data = df_sorted[train_idx]
            val_data = df_sorted[val_idx]
            splits.append((train_data, val_data))
            
        return splits
    
    def train_base_model(self, train_data: pl.DataFrame, 
                        val_data: pl.DataFrame,
                        verbose: bool = True) -> Dict:
        """
        Train Histogram-based Gradient Boosting model with validation
        
        Parameters:
        -----------
        train_data : pl.DataFrame
            Training data
        val_data : pl.DataFrame
            Validation data
        verbose : bool, default=True
            Whether to print training progress
            
        Returns:
        --------
        Dict
            Dictionary containing trained model and performance metrics
        """
        # Prepare data
        X_train = train_data.select(self.feature_cols).to_numpy()
        y_train = train_data[self.target_col].to_numpy()
        w_train = train_data['weight'].to_numpy()
        
        X_val = val_data.select(self.feature_cols).to_numpy()
        y_val = val_data[self.target_col].to_numpy()
        w_val = val_data['weight'].to_numpy()
        
        # Initialize model with optimized parameters
        model = HistGradientBoostingRegressor(
            max_iter=1000,
            learning_rate=0.05,
            max_depth=6,
            max_bins=255,  # More bins for better precision
            min_samples_leaf=20,
            l2_regularization=1.0,
            early_stopping=True,
            validation_fraction=0.2,
            n_iter_no_change=50,
            scoring='neg_mean_squared_error',  # More appropriate for financial data
            verbose=1 if verbose else 0,
            random_state=42
        )
        
        # Train model with sample weights
        if verbose:
            print("Training model...")
            
        model.fit(X_train, y_train, sample_weight=w_train)
        
        # Make predictions
        val_pred = model.predict(X_val)
        
        # Calculate weighted R2
        weighted_r2 = self.calculate_weighted_r2(y_val, val_pred, w_val)
        
        if verbose:
            print(f"Best iteration: {model.n_iter_}")
            print(f"Validation R²: {weighted_r2:.4f}")
        
        # Get feature importance (normalized)
        importance_dict = {}
        for idx, importance in enumerate(model.feature_importances_):
            importance_dict[self.feature_cols[idx]] = importance
        
        return {
            'model': model,
            'validation_score': weighted_r2,
            'feature_importance': importance_dict,
            'best_iteration': model.n_iter_
        }
    
    def calculate_weighted_r2(self, y_true: np.ndarray, 
                            y_pred: np.ndarray, 
                            weights: np.ndarray) -> float:
        """
        Calculate weighted R² score
        """
        numerator = np.sum(weights * (y_true - y_pred) ** 2)
        denominator = np.sum(weights * y_true ** 2)
        r2 = 1 - numerator / denominator
        return float(r2)  # Ensure Python float is returned
    
    def analyze_predictions(self, model_dict: Dict, 
                          val_data: pl.DataFrame) -> Dict:
        """
        Analyze model predictions across different segments
        """
        # Make predictions
        X_val = val_data.select(self.feature_cols).to_numpy()
        val_pred = model_dict['model'].predict(X_val)
        y_val = val_data[self.target_col].to_numpy()
        
        # Analysis by market regime
        regimes = val_data['vol_regime'].to_numpy()
        regime_scores = {}
        
        for regime in ['high', 'normal', 'low']:
            mask = regimes == regime
            if mask.any():
                regime_scores[regime] = self.calculate_weighted_r2(
                    y_val[mask], 
                    val_pred[mask], 
                    val_data['weight'].to_numpy()[mask]
                )
        
        # Analysis by symbol
        symbols = val_data['symbol_id'].to_numpy()
        symbol_scores = {}
        
        for symbol in np.unique(symbols):
            mask = symbols == symbol
            symbol_scores[symbol] = self.calculate_weighted_r2(
                y_val[mask], 
                val_pred[mask], 
                val_data['weight'].to_numpy()[mask]
            )
        
        # Get top features
        top_features = sorted(
            model_dict['feature_importance'].items(),
            key=lambda x: x[1],
            reverse=True
        )[:10]
        
        return {
            'regime_performance': regime_scores,
            'symbol_performance': symbol_scores,
            'overall_score': model_dict['validation_score'],
            'best_iteration': model_dict['best_iteration'],
            'top_features': top_features
        }
    
    def plot_analysis(self, analysis_dict: Dict):
        """
        Plot comprehensive model analysis results
        """
        fig, axes = plt.subplots(2, 2, figsize=(15, 12))
        
        # Plot regime performance
        regime_scores = analysis_dict['regime_performance']
        bars1 = axes[0,0].bar(regime_scores.keys(), regime_scores.values())
        axes[0,0].set_title('Performance by Market Regime')
        axes[0,0].set_ylabel('R² Score')
        axes[0,0].grid(True, alpha=0.3)
        
        # Add value labels on bars
        for bar in bars1:
            height = bar.get_height()
            axes[0,0].text(bar.get_x() + bar.get_width()/2., height,
                         f'{height:.4f}',
                         ha='center', va='bottom')
        
        # Plot symbol performance
        symbol_scores = analysis_dict['symbol_performance']
        bars2 = axes[0,1].bar(symbol_scores.keys(), symbol_scores.values())
        axes[0,1].set_title('Performance by Symbol')
        axes[0,1].set_ylabel('R² Score')
        axes[0,1].grid(True, alpha=0.3)
        axes[0,1].tick_params(axis='x', rotation=45)
        
        # Add value labels on bars
        for bar in bars2:
            height = bar.get_height()
            axes[0,1].text(bar.get_x() + bar.get_width()/2., height,
                         f'{height:.4f}',
                         ha='center', va='bottom')
        
        # Plot top features
        top_features = analysis_dict['top_features']
        features, importances = zip(*top_features)
        y_pos = np.arange(len(features))
        bars3 = axes[1,0].barh(y_pos, importances)
        axes[1,0].set_yticks(y_pos)
        axes[1,0].set_yticklabels([f[:20] + '...' if len(f) > 20 else f 
                                  for f in features])
        axes[1,0].set_title('Top Feature Importance')
        axes[1,0].grid(True, alpha=0.3)
        
        # Add value labels on bars
        for bar in bars3:
            width = bar.get_width()
            axes[1,0].text(width, bar.get_y() + bar.get_height()/2.,
                         f'{width:.4f}',
                         ha='left', va='center')
        
        # Add performance summary
        summary_text = (
            f"Overall R² Score: {analysis_dict['overall_score']:.4f}\n"
            f"Best Iteration: {analysis_dict['best_iteration']}\n"
            f"Features Used: {len(self.feature_cols)}\n"
            f"Top Feature: {top_features[0][0]}\n"
            f"Top Feature Importance: {top_features[0][1]:.4f}"
        )
        axes[1,1].text(0.5, 0.5, summary_text,
                      horizontalalignment='center',
                      verticalalignment='center',
                      fontsize=12,
                      bbox=dict(facecolor='white', alpha=0.8))
        axes[1,1].axis('off')
        
        plt.tight_layout()
        plt.show()

if __name__ == "__main__":
    print("Loading engineered features...")
    try:
        df = pl.read_parquet("../data/processed/engineered_features.parquet")
    except Exception as e:
        print(f"Error loading data: {e}")
        raise
    
    print("\nInitializing model development...")
    model_dev = ModelDevelopment(df)
    
    print("\nCreating validation splits...")
    splits = model_dev.prepare_validation_splits()
    
    print("\nTraining model...")
    results = []
    for i, (train_data, val_data) in enumerate(splits, 1):
        print(f"\nFold {i}:")
        print(f"Training samples: {len(train_data)}")
        print(f"Validation samples: {len(val_data)}")
        
        model_dict = model_dev.train_base_model(train_data, val_data)
        analysis = model_dev.analyze_predictions(model_dict, val_data)
        results.append(analysis)
        print(f"Best iteration: {analysis['best_iteration']}")
        print(f"Validation R²: {analysis['overall_score']:.4f}")
        
        print("\nPlotting analysis...")
        model_dev.plot_analysis(analysis)
    
    print("\n=== Model Development Summary ===")
    avg_score = np.mean([r['overall_score'] for r in results])
    std_score = np.std([r['overall_score'] for r in results])
    print(f"Average R² score: {avg_score:.4f} ± {std_score:.4f}")
    
    best_fold = np.argmax([r['overall_score'] for r in results]) + 1
    print(f"\nBest model from fold {best_fold}")
    best_result = results[best_fold - 1]
    print(f"Score: {best_result['overall_score']:.4f}")
    print(f"Best iteration: {best_result['best_iteration']}")
    print("\nTop features:")
    for feature, importance in best_result['top_features']:
        print(f"{feature}: {importance:.4f}")

Loading engineered features...

Initializing model development...
Identified 71 features

Creating validation splits...

Training model...

Fold 1:
Training samples: 791412
Validation samples: 791409
Training model...
Binning 0.360 GB of training data: 1.591 s
Binning 0.090 GB of validation data: 0.127 s
Fitting gradient boosted rounds:
[1/1000] 1 tree, 31 leaves, max depth = 6, train score: -0.57385, val score: -0.55158, in 0.069s
[2/1000] 1 tree, 31 leaves, max depth = 6, train score: -0.57279, val score: -0.55077, in 0.083s
[3/1000] 1 tree, 31 leaves, max depth = 6, train score: -0.57206, val score: -0.55007, in 0.071s
[4/1000] 1 tree, 31 leaves, max depth = 6, train score: -0.57135, val score: -0.54938, in 0.081s
[5/1000] 1 tree, 31 leaves, max depth = 6, train score: -0.57065, val score: -0.54881, in 0.087s
[6/1000] 1 tree, 31 leaves, max depth = 6, train score: -0.56986, val score: -0.54825, in 0.087s
[7/1000] 1 tree, 31 leaves, max depth = 6, train score: -0.56906, val score: -0

AttributeError: 'HistGradientBoostingRegressor' object has no attribute 'feature_importances_'